In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf
# import scipy.optimize as sco
import plotly.graph_objects as go
import plotly.express as px
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
import matplotlib.pyplot as plt
import matplotlib.style as style
%matplotlib inline
plt.style.use('seaborn') 
yf.pdr_override()

import warnings
warnings.filterwarnings('ignore')


C:\Users\ssora\AppData\Local\Temp\ipykernel_10512\166870695.py:17: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
# Import data function
def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']
    stockData = stockData.round(2)
    stockData.dropna(inplace=True)
    stockData.reset_index(inplace=True)
    stockData.set_index("Date", inplace=True)

    return stockData

In [3]:
ETF_list = [
    'JPST',
    'BND', 'BNDX', 'JNK',
    'VT', 'VOO', 'VGK', 'IEMG', 'HEWJ',
    'IGM', 'IXJ', 'IWQU.L',
    'REET', 'IGF', 'PDBC', 'GLD'
]

ETF_map = {
    'JPST': 'JPMorgan Ultra-Short Income ETF',
    'BND': 'Vanguard Total Bond Market ETF',
    'BNDX': 'Vanguard Total International Bond ETF',
    'JNK': 'SPDR Bloomberg Barclays High Yield Bond ETF',
    'VT': 'Vanguard Total World Stock ETF',
    'VOO': 'Vanguard S&P 500 ETF',
    'VGK': 'Vanguard FTSE Europe ETF',
    'IEMG': 'iShares Core MSCI Emerging Markets ETF',
    'HEWJ': 'iShares Currency Hedged MSCI Japan ETF',
    'IGM': 'iShares Expanded Tech Sector ETF',
    'IXJ': 'iShares Global Healthcare ETF',
    'IWQU.L': 'iShares Edge MSCI World Quality Factor UCITS ETF',
    'REET': 'iShares Global REIT ETF',
    'IGF': 'iShares Global Infrastructure ETF',
    'PDBC': 'Invesco Optimum Yield Diversified Commodity Strategy No K-1 ETF',
    'GLD': 'SPDR Gold Shares'
    }


# 5 years data
startDate = dt.datetime(2018, 1, 1)
endDate = dt.datetime(2022, 12, 31)

df_getData = getData(ETF_list, startDate, endDate)
df_getData.head()

[*********************100%***********************]  16 of 16 completed


,BND,BNDX,GLD,HEWJ,IEMG,IGF,IGM,IWQU.L,IXJ,JNK,JPST,PDBC,REET,VGK,VOO,VT
Date,,,,,,,,,,,,,,,,
2018-01-02,81.34,54.25,125.15,33.42,58.00,45.48,171.10,34.14,56.87,110.28,50.05,17.55,26.05,59.54,247.09,74.91
2018-01-03,81.41,54.29,124.82,33.97,58.51,45.48,173.20,34.31,57.37,110.82,50.09,17.65,26.09,59.75,248.56,75.42
2018-01-04,81.37,54.30,125.46,34.46,58.75,45.79,174.08,34.55,57.44,110.97,50.06,17.66,25.73,60.45,249.65,75.87
2018-01-05,81.26,54.30,125.33,34.65,59.25,45.92,175.95,34.71,58.03,111.09,50.08,17.59,25.78,60.86,251.25,76.31
2018-01-08,81.27,54.31,125.31,34.86,59.29,45.87,177.09,34.79,57.80,110.97,50.09,17.58,25.82,60.69,251.77,76.39


In [4]:
df_getData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1234 entries, 2018-01-02 to 2022-12-30
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BND     1234 non-null   float64
 1   BNDX    1234 non-null   float64
 2   GLD     1234 non-null   float64
 3   HEWJ    1234 non-null   float64
 4   IEMG    1234 non-null   float64
 5   IGF     1234 non-null   float64
 6   IGM     1234 non-null   float64
 7   IWQU.L  1234 non-null   float64
 8   IXJ     1234 non-null   float64
 9   JNK     1234 non-null   float64
 10  JPST    1234 non-null   float64
 11  PDBC    1234 non-null   float64
 12  REET    1234 non-null   float64
 13  VGK     1234 non-null   float64
 14  VOO     1234 non-null   float64
 15  VT      1234 non-null   float64
dtypes: float64(16)
memory usage: 163.9 KB


# Long Position Only

In [5]:
# risk free rate
risk_free = 0.02

###### portfolio type ######
long_only = tuple([0,1])
long_short = tuple([-1,1])

port_type = long_only # edit here


###### Adding Constrains ######

# Asset Mapping
asset_map = {
    'VT': 'Equity',
    'VOO': 'Equity',
    'VGK': 'Equity',
    'IEMG': 'Equity',
    'HEWJ': 'Equity',
    'IGM': 'Equity',
    'IXJ': 'Equity',
    'IWQU.L': 'Equity',
    'BND': 'Fixed Income',
    'BNDX': 'Fixed Income',
    'JNK': 'Fixed Income',
    'REET': 'Alternatives',
    'IGF': 'Alternatives',
    'PDBC': 'Alternatives',
    'GLD': 'Alternatives',
    'JPST': 'Cash/Equivalent'
}

### Fixed Income Portfolio ###
asset_lower_fixed = {
    'Cash/Equivalent': 0.0,
    'Fixed Income': 0.4,
    'Equity': 0.0,
    'Alternatives': 0.0}
asset_upper_fixed = {
    'Cash/Equivalent': 0.5,
    'Fixed Income': 1,
    'Equity': 0.0,
    'Alternatives': 0.0}

### Moderate Portfolio ###
asset_lower_moderate = {
    'Cash/Equivalent': 0.0,
    'Fixed Income': 0.25,
    'Equity': 0.1,
    'Alternatives': 0.0}
asset_upper_moderate = {
    'Cash/Equivalent': 0.5,
    'Fixed Income': 0.8,
    'Equity': 0.3,
    'Alternatives': 0.2}

### Balanced Portfolio ###
asset_lower_balanced = {
    'Cash/Equivalent': 0.0,
    'Fixed Income': 0.05,
    'Equity': 0.3,
    'Alternatives': 0.0}
asset_upper_balanced = {
    'Cash/Equivalent': 0.5,
    'Fixed Income': 0.6,
    'Equity': 0.6,
    'Alternatives': 0.3}

### Aggressive Portfolio ###
asset_lower_aggressive = {
    'Cash/Equivalent': 0.0,
    'Fixed Income': 0.0,
    'Equity': 0.55,
    'Alternatives': 0.0}
asset_upper_aggressive = {
    'Cash/Equivalent': 0.4,
    'Fixed Income': 0.3,
    'Equity': 0.9,
    'Alternatives': 0.3}

### Equity Portfolio ###
asset_lower_equity = {
    'Cash/Equivalent': 0.0,
    'Fixed Income': 0.0,
    'Equity': 0.7,
    'Alternatives': 0.0}
asset_upper_equity = {
    'Cash/Equivalent': 0.3,
    'Fixed Income': 0.0,
    'Equity': 1,
    'Alternatives': 0.0}

# create new dataframe for each portfolio

In [6]:
# filter asset class based on portfolio type 
df = df_getData.copy()

# filter column based on asset_map (Cash/Equivalent, Fixed Income, Equity, Alternatives)
df_cash = df[df.columns[df.columns.isin([k for k,v in asset_map.items() if v == 'Cash/Equivalent'])]]
df_fixed = df[df.columns[df.columns.isin([k for k,v in asset_map.items() if v == 'Fixed Income'])]]
df_equity = df[df.columns[df.columns.isin([k for k,v in asset_map.items() if v == 'Equity'])]]
df_alternatives = df[df.columns[df.columns.isin([k for k,v in asset_map.items() if v == 'Alternatives'])]]

In [7]:
### Fixed Income Portfolio ###
df_cash_fixed = pd.concat([df_cash, df_fixed], axis=1)

### Moderate/Balanced/Aggressive Portfolio ###
df_mba = df_getData.copy()

### Equity Portfolio ###
df_equity = pd.concat([df_equity, df_cash], axis=1)

# 1. Fixed Income Portfolio

In [8]:
# Calculate expected returns and sample covariance
df = df_cash_fixed.copy()
expected_returns1 = expected_returns.mean_historical_return(df)
cov_matrix1 = risk_models.sample_cov(df)

ef_fixed = EfficientFrontier(expected_returns1, cov_matrix1, weight_bounds=port_type)
ef_fixed.add_sector_constraints(asset_map, asset_lower_fixed, asset_upper_fixed)

# find the minimum volatility portfolio
weights_fixed = ef_fixed.min_volatility()
cleaned_weights_fixed = ef_fixed.clean_weights()

df_weights_fixed = pd.DataFrame.from_dict(cleaned_weights_fixed, orient='index', columns=['FI_Weight'])
df_weights_fixed['FI_Weight'] = df_weights_fixed['FI_Weight'].apply(lambda x: round(x*100, 2))

ef_fixed.portfolio_performance(verbose=True)
display(df_weights_fixed)

Expected annual return: -1.3%
Annual volatility: 2.6%
Sharpe Ratio: -1.30


,FI_Weight
JPST,50.00
BND,2.63
BNDX,46.09
JNK,1.28


# 2. Moderate Portfolio

In [9]:
# Calculate expected returns and sample covariance
df = df_mba.copy()
expected_returns2 = expected_returns.mean_historical_return(df)
cov_matrix2 = risk_models.sample_cov(df)

ef_moderate = EfficientFrontier(expected_returns2, cov_matrix2, weight_bounds=port_type)
ef_moderate.add_sector_constraints(asset_map, asset_lower_moderate, asset_upper_moderate)
ef_moderate.add_constraint(lambda x: x >= 0.01)

# find the maximum Sharpe ratio portfolio
weights_moderate = ef_moderate.max_sharpe(risk_free_rate=risk_free)
cleaned_weights_moderate = ef_moderate.clean_weights()

df_weights_moderate = pd.DataFrame.from_dict(cleaned_weights_moderate, orient='index', columns=['Mod_Weight'])
df_weights_moderate['Mod_Weight'] = df_weights_moderate['Mod_Weight'].apply(lambda x: round(x*100, 2))

ef_moderate.portfolio_performance(verbose=True)
display(df_weights_moderate)

Expected annual return: 3.0%
Annual volatility: 9.4%
Sharpe Ratio: 0.10


,Mod_Weight
BND,23.0
BNDX,1.0
GLD,17.0
HEWJ,1.0
IEMG,1.0
IGF,1.0
IGM,23.0
IWQU.L,1.0
IXJ,1.0
JNK,1.0


# 3. Balanced Portfolio

In [10]:
ef_balanced = EfficientFrontier(expected_returns2, cov_matrix2, weight_bounds=port_type)
ef_balanced.add_sector_constraints(asset_map, asset_lower_balanced, asset_upper_balanced)
ef_balanced.add_constraint(lambda x: x >= 0.01)

# find the maximum sharpe ratio portfolio
weights_balanced = ef_balanced.max_sharpe(risk_free_rate=risk_free)
cleaned_weights_balanced = ef_balanced.clean_weights()

df_weights_balanced = pd.DataFrame.from_dict(cleaned_weights_balanced, orient='index', columns=['Bal_Weight'])
df_weights_balanced['Bal_Weight'] = df_weights_balanced['Bal_Weight'].apply(lambda x: round(x*100, 2))

ef_balanced.portfolio_performance(verbose=True)
display(df_weights_balanced)

Expected annual return: 6.3%
Annual volatility: 12.9%
Sharpe Ratio: 0.33


,Bal_Weight
BND,3.00
BNDX,1.00
GLD,27.00
HEWJ,1.00
IEMG,1.00
IGF,1.00
IGM,8.04
IWQU.L,4.70
IXJ,42.26
JNK,1.00


# 4. Aggressive Portfolio

In [11]:
ef_agg = EfficientFrontier(expected_returns2, cov_matrix2, weight_bounds=port_type)
ef_agg.add_sector_constraints(asset_map, asset_lower_aggressive, asset_upper_aggressive)
ef_agg.add_constraint(lambda x: x >= 0.01)

# find the maximum sharpe ratio portfolio
weights_agg = ef_agg.max_sharpe(risk_free_rate=risk_free)
cleaned_weights_agg = ef_agg.clean_weights()

df_weights_agg = pd.DataFrame.from_dict(cleaned_weights_agg, orient='index', columns=['Agg_Weight'])
df_weights_agg['Agg_Weight'] = df_weights_agg['Agg_Weight'].apply(lambda x: round(x*100, 2))

ef_agg.portfolio_performance(verbose=True)
display(df_weights_agg)

Expected annual return: 6.8%
Annual volatility: 13.6%
Sharpe Ratio: 0.35


,Agg_Weight
BND,1.00
BNDX,1.00
GLD,27.00
HEWJ,1.00
IEMG,1.00
IGF,1.00
IGM,7.09
IWQU.L,7.31
IXJ,46.60
JNK,1.00


# 5. Equity Portfolio

In [12]:
# Calculate expected returns and sample covariance
df = df_equity.copy()
expected_returns5 = expected_returns.mean_historical_return(df)
cov_matrix5 = risk_models.sample_cov(df)

ef_equity = EfficientFrontier(expected_returns5, cov_matrix5, weight_bounds=port_type)
ef_equity.add_sector_constraints(asset_map, asset_lower_equity, asset_upper_equity)
ef_equity.add_constraint(lambda x: x >= 0.01)

# find the maximum sharpe ratio portfolio
weights_equity = ef_equity.max_sharpe(risk_free_rate=risk_free)
cleaned_weights_equity = ef_equity.clean_weights()

df_weights_equity = pd.DataFrame.from_dict(cleaned_weights_equity, orient='index', columns=['Eq_Weight'])
df_weights_equity['Eq_Weight'] = df_weights_equity['Eq_Weight'].apply(lambda x: round(x*100, 2))

ef_equity.portfolio_performance(verbose=True)
display(df_weights_equity)

Expected annual return: 8.1%
Annual volatility: 18.0%
Sharpe Ratio: 0.34


,Eq_Weight
HEWJ,1.00
IEMG,1.00
IGM,12.28
IWQU.L,12.53
IXJ,69.19
VGK,1.00
VOO,1.00
VT,1.00
JPST,1.00


# Summary

In [13]:
# concat all portfolio weights
df_weights = pd.concat([df_weights_fixed, df_weights_moderate, df_weights_balanced, df_weights_agg, df_weights_equity], axis=1)
df_weights.replace(np.nan, '-', inplace=True)

# ETF_map to ETF_name
df_weights['ETF_Name'] = df_weights.index.map(ETF_map)

# map asset class to portfolio
df_weights['Asset_Class'] = df_weights.index.map(asset_map)
# sort asset class 1.Cash/Equivalent, 2.Fixed Income, 3.Equity, 4.Alternatives
df_weights['Asset_Class'] = pd.Categorical(df_weights['Asset_Class'], ['Cash/Equivalent', 'Fixed Income', 'Equity', 'Alternatives'])
df_weights.sort_values(by=['Asset_Class'], inplace=True)

df_weights = df_weights[['ETF_Name', 'Asset_Class', 'FI_Weight', 'Mod_Weight', 'Bal_Weight', 'Agg_Weight', 'Eq_Weight']]

df_weights

,ETF_Name,Asset_Class,FI_Weight,Mod_Weight,Bal_Weight,Agg_Weight,Eq_Weight
JPST,JPMorgan Ultra-Short Income ETF,Cash/Equivalent,50.0,25.0,5.00,1.00,1.0
BND,Vanguard Total Bond Market ETF,Fixed Income,2.63,23.0,3.00,1.00,-
BNDX,Vanguard Total International Bond ETF,Fixed Income,46.09,1.0,1.00,1.00,-
JNK,SPDR Bloomberg Barclays High Yield Bond ETF,Fixed Income,1.28,1.0,1.00,1.00,-
HEWJ,iShares Currency Hedged MSCI Japan ETF,Equity,-,1.0,1.00,1.00,1.0
IEMG,iShares Core MSCI Emerging Markets ETF,Equity,-,1.0,1.00,1.00,1.0
IGM,iShares Expanded Tech Sector ETF,Equity,-,23.0,8.04,7.09,12.28
IWQU.L,iShares Edge MSCI World Quality Factor UCITS ETF,Equity,-,1.0,4.70,7.31,12.53
IXJ,iShares Global Healthcare ETF,Equity,-,1.0,42.26,46.60,69.19
VGK,Vanguard FTSE Europe ETF,Equity,-,1.0,1.00,1.00,1.0


In [14]:
df_weights.replace('-', np.nan, inplace=True)
# df_weights group by asset class
df_weights_asset = df_weights.groupby('Asset_Class').sum()
df_weights_asset

,FI_Weight,Mod_Weight,Bal_Weight,Agg_Weight,Eq_Weight
Asset_Class,,,,,
Cash/Equivalent,50.0,25.0,5.0,1.0,1.0
Fixed Income,50.0,25.0,5.0,3.0,0.0
Equity,0.0,30.0,60.0,66.0,99.0
Alternatives,0.0,20.0,30.0,30.0,0.0


In [15]:
# Return, Risk, Sharpe Ratio for each portfolio to dataframe
df_performance = pd.DataFrame(columns=['Portfolio', 'Expected Annual Return (%)', 'Annual volatility (%)', 'Sharpe Ratio'])

# Fixed Income
df_performance.loc[0] = ['Fixed Income', ef_fixed.portfolio_performance()[0], ef_fixed.portfolio_performance()[1], ef_fixed.portfolio_performance()[2]]
# Moderate
df_performance.loc[1] = ['Moderate', ef_moderate.portfolio_performance()[0], ef_moderate.portfolio_performance()[1], ef_moderate.portfolio_performance()[2]]
# Balanced
df_performance.loc[2] = ['Balanced', ef_balanced.portfolio_performance()[0], ef_balanced.portfolio_performance()[1], ef_balanced.portfolio_performance()[2]]
# Aggressive
df_performance.loc[3] = ['Aggressive', ef_agg.portfolio_performance()[0], ef_agg.portfolio_performance()[1], ef_agg.portfolio_performance()[2]]
# Equity
df_performance.loc[4] = ['Equity', ef_equity.portfolio_performance()[0], ef_equity.portfolio_performance()[1], ef_equity.portfolio_performance()[2]]

df_performance = df_performance.round(4)

# change Expected Annual Return (%) and Annual volatility to percentage
df_performance['Expected Annual Return (%)'] = df_performance['Expected Annual Return (%)'].apply(lambda x: round(x*100, 2))
df_performance['Annual volatility (%)'] = df_performance['Annual volatility (%)'].apply(lambda x: round(x*100, 2))

df_performance


,Portfolio,Expected Annual Return (%),Annual volatility (%),Sharpe Ratio
0,Fixed Income,-1.35,2.58,-1.2999
1,Moderate,2.98,9.38,0.1045
2,Balanced,6.30,12.85,0.3349
3,Aggressive,6.80,13.65,0.3517
4,Equity,8.05,18.05,0.3354


# 2023 Performance

In [16]:
# get new data 
startDate2 = dt.datetime(2023, 1, 1)
endDate2 = dt.datetime(2023, 4, 30)

df2023 = getData(ETF_list, startDate2, endDate2)
df2023.head()

[*********************100%***********************]  16 of 16 completed


,BND,BNDX,GLD,HEWJ,IEMG,IGF,IGM,IWQU.L,IXJ,JNK,JPST,PDBC,REET,VGK,VOO,VT
Date,,,,,,,,,,,,,,,,
2023-01-03,72.22,47.85,171.06,26.23,47.07,45.83,279.81,46.99,84.92,90.18,50.12,14.41,22.66,56.14,349.99,86.24
2023-01-04,72.63,48.05,172.67,26.33,48.44,46.50,281.93,47.54,85.18,91.29,50.14,14.12,23.10,57.21,352.51,87.30
2023-01-05,72.55,47.99,170.52,26.12,48.20,46.33,275.98,46.94,84.31,91.12,50.15,14.06,22.60,56.69,348.66,86.40
2023-01-06,73.35,48.30,173.71,26.52,49.24,47.31,283.71,47.89,85.26,92.44,50.17,14.15,23.06,58.20,356.59,88.38
2023-01-09,73.55,48.18,174.10,26.52,49.59,47.61,287.32,48.74,84.15,92.84,50.18,14.35,23.10,58.57,356.33,88.59


In [17]:
df2023.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80 entries, 2023-01-03 to 2023-04-28
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BND     80 non-null     float64
 1   BNDX    80 non-null     float64
 2   GLD     80 non-null     float64
 3   HEWJ    80 non-null     float64
 4   IEMG    80 non-null     float64
 5   IGF     80 non-null     float64
 6   IGM     80 non-null     float64
 7   IWQU.L  80 non-null     float64
 8   IXJ     80 non-null     float64
 9   JNK     80 non-null     float64
 10  JPST    80 non-null     float64
 11  PDBC    80 non-null     float64
 12  REET    80 non-null     float64
 13  VGK     80 non-null     float64
 14  VOO     80 non-null     float64
 15  VT      80 non-null     float64
dtypes: float64(16)
memory usage: 10.6 KB


### 2023 ETF Portfolio Performance

In [18]:
# df2023 cumulative return
df2023_cum = df2023.copy()
df2023_cum = df2023_cum.pct_change()
df2023_cum = (df2023_cum+1).cumprod()
df2023_cum.head()

,BND,BNDX,GLD,HEWJ,IEMG,IGF,IGM,IWQU.L,IXJ,JNK,JPST,PDBC,REET,VGK,VOO,VT
Date,,,,,,,,,,,,,,,,
2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04,1.005677,1.004180,1.009412,1.003812,1.029106,1.014619,1.007577,1.011705,1.003062,1.012309,1.000399,0.979875,1.019417,1.019059,1.007200,1.012291
2023-01-05,1.004569,1.002926,0.996843,0.995806,1.024007,1.010910,0.986312,0.998936,0.992817,1.010424,1.000599,0.975711,0.997352,1.009797,0.996200,1.001855
2023-01-06,1.015647,1.009404,1.015492,1.011056,1.046102,1.032293,1.013938,1.019153,1.004004,1.025061,1.000998,0.981957,1.017652,1.036694,1.018858,1.024814
2023-01-09,1.018416,1.006897,1.017772,1.011056,1.053537,1.038839,1.026840,1.037242,0.990933,1.029497,1.001197,0.995836,1.019417,1.043285,1.018115,1.027250


In [19]:
# df2023 cumulative return plot
fig = go.Figure()
for col in df2023_cum.columns:
    fig.add_trace(go.Scatter(x=df2023_cum.index, y=df2023_cum[col], name=col))

fig.update_layout(title='Cumulative Return of ETFs from 2023/1/1 to 2023/4/30', xaxis_title='Date', yaxis_title='Cumulative Return')
fig.show()


### 2023 Portfolio Performace

In [20]:
df2023['Fixed Income'] = 0
df2023['Moderate'] = 0
df2023['Balanced'] = 0
df2023['Aggressive'] = 0
df2023['Equity'] = 0

# calculate portfolio value
for ticker, weight in cleaned_weights_fixed.items():
    df2023['Fixed Income'] += df2023[ticker] * weight

for ticker, weight in cleaned_weights_moderate.items():
    df2023['Moderate'] += df2023[ticker] * weight

for ticker, weight in cleaned_weights_balanced.items():
    df2023['Balanced'] += df2023[ticker] * weight

for ticker, weight in cleaned_weights_agg.items():
    df2023['Aggressive'] += df2023[ticker] * weight

for ticker, weight in cleaned_weights_equity.items():
    df2023['Equity'] += df2023[ticker] * weight

df2023_port = df2023[['Fixed Income', 'Moderate', 'Balanced', 'Aggressive', 'Equity']]
df2023_port

,Fixed Income,Moderate,Balanced,Aggressive,Equity
Date,,,,,
2023-01-03,50.166793,131.7622,119.307933,118.124056,105.155617
2023-01-04,50.293934,132.7134,120.144544,118.957236,105.726090
2023-01-05,50.267004,130.8820,118.621316,117.438051,104.261391
2023-01-06,50.457793,133.5704,120.735157,119.528011,106.115514
2023-01-09,50.417854,134.5278,120.722855,119.451135,105.903918
...,...,...,...,...,...
2023-04-24,50.606449,147.4459,129.237257,127.732651,114.569783
2023-04-25,50.812064,145.8139,128.435132,126.951896,112.942436
2023-04-26,50.736879,146.0729,127.825973,126.269589,112.398288


In [21]:
# daily percentage change
df2023_pct = df2023_port.pct_change()
df2023_pct = df2023_pct

# plot daily percentage change
fig = go.Figure()
for col in df2023_pct.columns:
    fig.add_trace(go.Scatter(x=df2023_pct.index, y=df2023_pct[col], mode='lines', name=col))
fig.update_layout(title='Daily Percentage Change (%)', xaxis_title='Date', yaxis_title='Percentage Change')
fig.update_yaxes(tickformat=".2%")
fig.show()

In [22]:
# daily percentage change standard deviation
df2023_pct.std()

Fixed Income    0.002202
Moderate        0.008417
Balanced        0.006519
Aggressive      0.006528
Equity          0.008745
dtype: float64

In [23]:
# cumulative return
df2023_cum = (df2023_pct + 1).cumprod()

# fill 1st row with 1
df2023_cum.iloc[0] = 1
df2023_cum.head()


,Fixed Income,Moderate,Balanced,Aggressive,Equity
Date,,,,,
2023-01-03,1.000000,1.000000,1.000000,1.000000,1.000000
2023-01-04,1.002534,1.007219,1.007012,1.007053,1.005425
2023-01-05,1.001998,0.993320,0.994245,0.994193,0.991496
2023-01-06,1.005801,1.013723,1.011963,1.011885,1.009128
2023-01-09,1.005005,1.020989,1.011859,1.011235,1.007116


In [24]:
# plot cumulative return
fig = go.Figure()
for col in df2023_cum.columns:
    fig.add_trace(go.Scatter(x=df2023_cum.index, y=df2023_cum[col], mode='lines', name=col))
fig.update_layout(title='2023 Portfolio Performance', xaxis_title='Date', yaxis_title='Portfolio Value')
# fig.update_yaxes(tickformat=".2%")
fig.show()

In [25]:
# df2023_cum.iloc[-1] and df2023_pct to dataframe
df_port_perf = pd.concat([df2023_cum.iloc[-1], df2023_pct.std()], axis=1)
df_port_perf.columns = ['Return (%)', 'Std (%)']
df_port_perf = df_port_perf.round(4)
df_port_perf['Return (%)'] = ((df_port_perf['Return (%)']-1)*100).round(2)
df_port_perf['Std (%)'] = (df_port_perf['Std (%)']*100)
df_port_perf

,Return (%),Std (%)
Fixed Income,1.40,0.22
Moderate,12.82,0.84
Balanced,8.27,0.65
Aggressive,7.99,0.65
Equity,8.77,0.87


In [26]:
df_weights

,ETF_Name,Asset_Class,FI_Weight,Mod_Weight,Bal_Weight,Agg_Weight,Eq_Weight
JPST,JPMorgan Ultra-Short Income ETF,Cash/Equivalent,50.00,25.0,5.00,1.00,1.00
BND,Vanguard Total Bond Market ETF,Fixed Income,2.63,23.0,3.00,1.00,NaN
BNDX,Vanguard Total International Bond ETF,Fixed Income,46.09,1.0,1.00,1.00,NaN
JNK,SPDR Bloomberg Barclays High Yield Bond ETF,Fixed Income,1.28,1.0,1.00,1.00,NaN
HEWJ,iShares Currency Hedged MSCI Japan ETF,Equity,NaN,1.0,1.00,1.00,1.00
IEMG,iShares Core MSCI Emerging Markets ETF,Equity,NaN,1.0,1.00,1.00,1.00
IGM,iShares Expanded Tech Sector ETF,Equity,NaN,23.0,8.04,7.09,12.28
IWQU.L,iShares Edge MSCI World Quality Factor UCITS ETF,Equity,NaN,1.0,4.70,7.31,12.53
IXJ,iShares Global Healthcare ETF,Equity,NaN,1.0,42.26,46.60,69.19
VGK,Vanguard FTSE Europe ETF,Equity,NaN,1.0,1.00,1.00,1.00


In [27]:
df_weights.replace('-', np.nan, inplace=True)
# df_weights group by asset class
df_weights_asset = df_weights.groupby('Asset_Class').sum()
df_weights_asset

,FI_Weight,Mod_Weight,Bal_Weight,Agg_Weight,Eq_Weight
Asset_Class,,,,,
Cash/Equivalent,50.0,25.0,5.0,1.0,1.0
Fixed Income,50.0,25.0,5.0,3.0,0.0
Equity,0.0,30.0,60.0,66.0,99.0
Alternatives,0.0,20.0,30.0,30.0,0.0
